In [1]:
!pip install nbformat


[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import polars as pl
import pandas as pd
import numpy as np
import plotly as plt
from plotly import graph_objects as go

df = pl.read_csv("META.csv")
df = df.with_columns([((pl.col("Close/Last").str.replace_all(r"\$","")).cast(pl.Float64)),
    ((pl.col("High").str.replace_all(r"\$","")).cast(pl.Float64)),
    ((pl.col("Low").str.replace_all(r"\$","")).cast(pl.Float64)),
    ((pl.col("Open").str.replace_all(r"\$","")).cast(pl.Float64))
])
df = df.reverse()
df

Date,Close/Last,Volume,Open,High,Low
str,f64,i64,f64,f64,f64
"""07/21/2015""",98.39,39163900,98.95,99.24,97.1427
"""07/22/2015""",97.04,28196110,96.74,97.58,95.92
"""07/23/2015""",95.44,29281230,96.96,97.45,94.81
"""07/24/2015""",96.95,33351170,97.35,97.76,95.88
"""07/27/2015""",94.17,38428000,96.58,96.61,93.8301
…,…,…,…,…,…
"""07/14/2025""",720.92,8939399,717.6,728.0,716.547
"""07/15/2025""",710.39,11529510,723.9,724.4716,709.82
"""07/16/2025""",702.91,13067630,713.37,713.97,699.2701


In [3]:
graph = go.Figure(data = [go.Candlestick(x=df["Date"],open=df["Open"],close=df["Close/Last"],high = df["High"],low=df["Low"],name="candlestick")])
graph.show()

In [75]:
from indicator import *
class Signal:
    def __init__(self,data,period,bollinger_range):
        self.dataframe = data
        self.period = period
        self.bollinger_range = bollinger_range
        self.fee_rate = 0.001
        
        
    def relative_strength_index(self):
        rsi = Relative_strength_index(self.dataframe["Close/Last"],period=self.period)
        return rsi
    
    def bollinger_bands(self):
        upper,lower,ma = bollinger(self.dataframe["Close/Last"],period=self.period,bollinger_range=self.bollinger_range)
        return upper,lower,ma
    
    def Average_True(self):
        return ATR_backtest(self.dataframe["Close/Last"],High=self.dataframe["High"],Low =self.dataframe["Low"],period = self.period)
    
    def relative_strength_index_signal(self,rsi):
        rsi_signal = [0]
        for i in range(1,len(rsi)):
            if rsi[i]>rsi[i-1]:
                if rsi[i]<30:
                    rsi_signal.append(2)
                elif rsi[i]>70:
                    rsi_signal.append(-1)
                else:
                    rsi_signal.append(0)
            elif rsi[i]>30 and rsi[i-1]<30:
                rsi_signal.append(1)
            elif rsi[i]>70 and rsi[i-1]<70:
                rsi_signal.append(-1)
            else:
                rsi_signal.append(0)
        return rsi_signal
    
    
    def bollinger_band_signal(self,upper_band,lower_band,moving_avg):
        bband_signal = [0]
        for i in range(1,len(upper_band)):
            if self.dataframe["Close/Last"][i] < moving_avg[i]:
                if self.dataframe["Close/Last"][i] > lower_band[i] and self.dataframe["Close/Last"][i-1] < lower_band[i-1]:
                    bband_signal.append(2)
                else:
                    bband_signal.append(1)
            elif self.dataframe["Close/Last"][i] < upper_band[i] and self.dataframe["Close/Last"][i-1] > upper_band[i-1]:
                bband_signal.append(-1)
            else:
                bband_signal.append(0)
        return bband_signal
    
    def rejection_candles_signal(self,Close,High,Open,Low):
        signal = rejection_candles(Close,High=High,Low=Low,Open=Open)
        return signal
        
    def buy(self,total,lot_size,current_price,buy_arr,signal):
        trade = lot_size * current_price
        fee = self.fee_rate*trade
        if trade + fee < total:
            total = total -trade-fee
            state = "buy"
            buy_arr.append(trade)
            signal.append(1)
        else:
            state="n/a"
            lot_size= 0
            signal.append(0)
        return total,state,lot_size,buy_arr,signal
        
    def sell(self,total,lot_size,price,signal,sell_arr):
        trade = price*lot_size
        fee = self.fee_rate*trade
        if fee < total + trade:
            total = total +trade -fee
            state = "n/a"
            signal.append(-1)
            sell_arr.append(trade)
        else:
            state = "buy"
            signal.append(0)
        return total,state,signal,sell_arr
    
    #needs validation to ensure it cannot be negative     
    def trades(self,rsi_signal,bband_signal,atr,rejection_signal,initial_money):
        total = initial_money
        total_arr = [total]
        state= "n/a"
        buys = []
        sells = []
        final_signal=[]
        relative_atr = atr/np.array(self.dataframe["Close/Last"])
        sma = pd.Series(self.dataframe["Close/Last"]).rolling(50).mean()
        sma = sma.fillna(0)
        fma = pd.Series(self.dataframe["Close/Last"]).rolling(5).mean()
        fma = fma.fillna(0)
        for i in range(1,len(rsi_signal)):
            current_price = self.dataframe["Close/Last"][i]
            current_position = 0
            
            if state == "n/a":
                if relative_atr[i-1]<0.04:
                    if (bband_signal[i-1]==2 or bband_signal[i-1]==1) or rejection_signal[i-1]==1:
                        risk = 0.02*total
                        if atr[i-1] == 0:
                            position_size = 0.0
                            final_signal.append(0)
                        else:
                            position_size = int(risk/(atr[i-1]*2))
                            max_alloc = 0.6
                            max_position = float((total * max_alloc) // current_price)
                            lot_size = min(position_size,max_position)
                            total,state,current_position,buys,final_signal = self.buy(total,lot_size,current_price,buys,final_signal)
                            buy_price = current_price
                    elif i>=30:
                        if fma[i-1]>sma[i-1]:
                            risk = 0.02*total
                            if atr[i-1] == 0:
                                position_size = 0.0
                                final_signal.append(0)
                            else:
                                position_size = int(risk/(atr[i-1]*2))
                                max_alloc = 0.4
                                max_position = float((total * max_alloc) // current_price)
                                lot_size = min(position_size,max_position)
                                total,state,current_position,buys,final_signal = self.buy(total,lot_size,current_price,buys,final_signal)
                                buy_price = current_price
                        else:
                            pass
                    
                        
                    else:
                        final_signal.append(0)
                elif relative_atr[i-1]<0.06:
                    if (bband_signal[i-1]==2 or bband_signal[i-1]==1)  and rejection_signal[i-1]==1:
                        #1% of total is an aggressive risk
                        risk = 0.02*total
                        if atr[i-1] == 0:
                            position_size = 0.0
                            final_signal.append(0)
                        else:
                            position_size = int(risk/(atr[i-1]*2))
                            max_alloc = 0.6
                            max_position = float((total * max_alloc) // current_price)
                            lot_size = min(position_size,max_position)
                            total,state,current_position,buys,final_signal = self.buy(total,lot_size,current_price,buys,final_signal)
                            buy_price = current_price
                    elif i>=30:
                        if fma[i-1]>sma[i-1]:
                            risk = 0.02*total
                            if atr[i-1] == 0:
                                position_size = 0.0
                                final_signal.append(0)
                            else:
                                position_size = int(risk/(atr[i-1]*2))
                                max_alloc = 0.4
                                max_position = float((total * max_alloc) // current_price)
                                lot_size = min(position_size,max_position)
                                total,state,current_position,buys,final_signal = self.buy(total,lot_size,current_price,buys,final_signal)
                                buy_price = current_price
                        else:
                            pass
                else:
                    final_signal.append(0)
            
            else:
                if relative_atr[i-1]>0.08:
                    position_size = current_position
                    total,state,final_signal,sells = self.sell(total,lot_size,current_price,final_signal,sells)
                elif (self.dataframe["Close/Last"][i] > buy_price + atr[i-1]*4) or self.dataframe["Close/Last"][i] < buy_price - atr[i-1]*2:
                    position_size = current_position
                    total,state,final_signal,sells = self.sell(total,lot_size,current_price,final_signal,sells)
                elif rejection_signal[i-1]==-1 or bband_signal[i-1]==-1:
                    position_size = current_position
                    total,state,final_signal,sells = self.sell(total,lot_size,current_price,final_signal,sells)
                    
                    
                else:
                    final_signal.append(0)
            total_arr.append(total)
        if state == "buy":
            position_size = current_position
            position_size = current_position
            total,state,final_signal,sells = self.sell(total,lot_size,current_price,final_signal,sells)
            total_arr.append(total)
            
        returns = np.diff(total_arr) / total_arr[:-1]  
        Sharpe_ratio = (np.mean(returns) / np.std(returns))
        stats_dict = {"Sharpe Ratio":Sharpe_ratio,
                      "Daily total": total_arr,
                      "rsi signal":rsi_signal,
                      "bollinger band signal": bband_signal,
                      "rejection signal": rejection_signal,
                      "overall signal":final_signal,
                      "bought":buys,
                      "sold":sells}
        return np.round(total,2),stats_dict
                
                        
                        
            
                    
    
    def backtest(self):
        rsi = self.relative_strength_index()
        ub,lb,ma = self.bollinger_bands()
        b_signal = self.bollinger_band_signal(ub,lb,ma)
        rs_signal = self.relative_strength_index_signal(rsi)
        average_true_range = self.Average_True()
        rejection_candles_sig = self.rejection_candles_signal(self.dataframe["Close/Last"],self.dataframe["High"],self.dataframe["Open"],self.dataframe["Low"])
        strat_total,stats_dict = self.trades(rs_signal,b_signal,average_true_range,rejection_candles_sig,100000)
        return strat_total,stats_dict
    
    

        
            
                
        

In [76]:
myStrat = Signal(df,5,2)
tot,stats = myStrat.backtest()
g = go.Figure(data=[go.Scatter(x=df["Date"][::10],y=stats["Daily total"][::10],mode="lines",name="returns"),
                    go.Scatter(x=df["Date"][::10],y=df["Close/Last"][::10],mode="lines",name="Daily Price")])
g.show()

In [70]:
print(stats["Daily total"][499])
checkStrat = Signal(df[:500],5,2)
check_total,check_stats = checkStrat.backtest()
print(check_stats["Daily total"][499])
g = go.Figure(data=[go.Scatter(x=df["Date"][:504],y=stats["Daily total"][:504],mode="lines",name="returns"),
                    go.Scatter(x=df["Date"][:504],y=check_stats["Daily total"][:504],mode="lines",name="Daily Price")])
g.show()

71378.65778000001
71378.65778000001


In [6]:
print(stats["bought"])

[19398.239999999998, 20630.97, 21476.29, 22249.469999999998, 22170.72, 25884.8, 26068.800000000003, 22558.140000000003, 24269.760000000002, 24669.7, 33588.17, 32456.22, 30659.2, 25658.49, 23397.44, 25194.0, 35094.6, 32549.579999999998, 33462.24, 45345.57, 45180.159999999996, 41988.6, 41119.65, 49293.200000000004, 48750.799999999996, 44505.240000000005, 45763.2, 45258.450000000004, 41991.840000000004, 39773.44, 36539.25, 45760.5, 45161.079999999994, 49572.0, 45062.899999999994, 39168.36, 40228.65, 46744.98, 52487.6, 60741.34, 54464.64, 54367.530000000006, 54150.72, 56503.439999999995, 56998.8, 47918.969999999994, 43526.79, 43347.450000000004, 57365.280000000006, 49975.0, 49879.86, 50886.5, 51475.28, 52187.200000000004, 45282.62, 33754.82, 62088.39, 54718.56, 54920.579999999994, 55535.5, 51104.48, 42350.65, 42027.34, 36790.299999999996, 40426.67999999999, 39451.6, 55601.28, 56680.31999999999, 67484.43, 56840.0, 53601.3, 54811.54, 50045.77, 54197.520000000004, 51446.01, 37554.020000000004

In [7]:
myStrat = Signal(df,5,1.5)
tot,stats = myStrat.backtest()
g = go.Figure(data=[go.Scatter(y=stats["overall signal"],mode="lines",name="returns")])
g.show()

In [8]:
f = go.Figure(data =[go.Scatter(y=np.array(stats["bought"])-np.array(stats["sold"]),mode="lines",name="bought")
                    ])
f.show()

In [9]:
def count_candle(dataframe,signals,closename,openname):
    total_bear = 0
    total_bull =0
    bear_up_count = 0
    bear_down_count = 0
    bull_up_count = 0
    bull_down_count = 0
    for i in range(0,len(signals)-1):
      if signals[i] == 1 or signals[i] == 2:
        total_bull += 1
        if dataframe[closename][i+1]> dataframe[openname][i+1]:
            bull_up_count +=1
        elif dataframe[closename][i+1]< dataframe[openname][i+1]:
             bull_down_count +=1
      elif signals[i] == -1:
        total_bear +=1
        if dataframe[closename][i+1]> dataframe[openname][i+1]:
            bear_up_count +=1
        elif dataframe[closename][i+1]< dataframe[openname][i+1]:
             bear_down_count +=1
      else:
        pass
    if total_bear ==0 and total_bull ==0:
      bear_acc,bull_acc = 0.0
    if total_bear ==0:
      bear_acc = 0.0
      bull_acc = (bull_up_count/total_bull)*100
    elif total_bull ==0:
      bull_acc = 0.0
      bear_acc = (bear_down_count/total_bear)*100
    else:
      bear_acc = (bear_down_count/total_bear)*100
      bull_acc = (bull_up_count/total_bull)*100
    return bear_acc,bull_acc

In [22]:
ovr_sig = stats["overall signal"]
bear_accuracy,bullish_accuracy = count_candle(df,ovr_sig,"Open","Close/Last")
print(bear_accuracy,bullish_accuracy)

47.27272727272727 41.07142857142857


In [23]:
rsi_sig = stats["rsi signal"]
print(rsi_sig)
rsi_bear_accuracy,rsi_bullish_accuracy = count_candle(df,rsi_sig,"Open","Close/Last")
print(rsi_bear_accuracy,rsi_bullish_accuracy)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, -1, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, -1, -1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, 0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, 0, 0, -1, -1, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, -1, -1, 0, -1, 0, 0, 2, -1, -1, -1, 0, -1, 0, 0, 0, -1, 0, 0, 0, 0, 0, -1, -1, 0, -1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 2, -1, -1, -1, -1, 0, 0, 0, 0, 2, 2, -1, 0, -1, 0, -1, -1, -1, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 2, 0, 0, -1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, -1, 0, -1, 0, 0, 0, 

In [24]:
bband_sig = stats["bollinger band signal"]

bband_bear_accuracy,bband_bullish_accuracy = count_candle(df,bband_sig,"Open","Close/Last")
print(bband_bear_accuracy,bband_bullish_accuracy)

100.0 48.23529411764706


In [25]:
reject_sig = stats["rejection signal"]
print(reject_sig)
bband_bear_accuracy,bband_bullish_accuracy = count_candle(df,reject_sig,"Open","Close/Last")
print(bband_bear_accuracy,bband_bullish_accuracy)

[0, 0, -1, 0, -1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 1, -1, -1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [74]:
print(stats["Sharpe Ratio"])
print(tot)

inf
243333.39


In [15]:
#benchmark
assets = int(100000/df["Close/Last"][-800])
returns = assets*df["Close/Last"][-1]
print(returns)

355661.39999999997


In [17]:
"""elif (rsi_signal[i-1]==1 or bband_signal[i-1]==1)and rejection_signal[i]==1:
                        risk = 0.01*total
                        if atr[i-1] == 0:
                            position_size = 0.0
                            final_signal.append(0)
                        else:
                            position_size = int(risk/(atr[i-1]*2))
                            max_alloc = 0.05
                            max_position = float((total * max_alloc) / self.dataframe["Close/Last"][i])
                            lot_size = min(position_size*self.dataframe["Close/Last"][i],max_position)
                            total,state,current_position,buys,final_signal = self.buy(total,lot_size,current_price,buys,final_signal)
                            buy_price = current_price
                        """
                        
                        elif relative_atr[i-1]<0.06:
                    if rsi_signal[i-1]==2 or bband_signal[i-1]==2 and rejection_signal[i]==1:
                        #1% of total is an aggressive risk
                        risk = 0.005*total
                        if atr[i-1] == 0:
                            position_size = 0.0
                            final_signal.append(0)
                        else:
                            position_size = int(risk/(atr[i-1]*2))
                            max_alloc = 0.05
                            max_position = float((total * max_alloc) / self.dataframe["Close/Last"][i])
                            lot_size = min(position_size*self.dataframe["Close/Last"][i],max_position)
                            total,state,current_position,buys,final_signal = self.buy(total,lot_size,current_price,buys,final_signal)
                            buy_price = current_price
                    else:
                        final_signal.append(0)

IndentationError: unindent does not match any outer indentation level (<string>, line 16)